In [1]:
import os
import sys
import numpy as np
import networkx as nx
import scipy
import requests

# We temporarily add the required programs to PATH so we can run them via the console.
# Ensure the install_dependencies.sh script has run successfully.
# Look at the README file if any issues occur in building RIVET.
os.environ["PATH"] += os.pathsep + os.getcwd() + "/dependencies/rivet/build/" 
os.environ["PATH"] += os.pathsep + os.getcwd() + "/dependencies/hera/geom_bottleneck/build/example/"

endings = ['251256', '261034', '291467', '281264', '351899', '361446', '401224']

In [2]:
# DOWNLOAD DATA
url = 'http://users.cecs.anu.edu.au/~bdm/data/'
for e in endings:
  r = requests.get(url + "sr" + e + ".g6", allow_redirects=True)
  filename = "./data/sr" + e + ".g6"
  os.makedirs(os.path.dirname(filename), exist_ok=True)
  open(filename, 'wb').write(r.content)

In [3]:
def build_filtration(vert_vals, G, name, xlabel, ylabel):
    """
    Input: vert_vals - A filtration function V -> R^2, inthe form of an  |V| x 2 matrix
            G - A networkx graph
            name - the name of the file to save the filtration
    Output: Clique complex + bifiltration
    """

    # Do vertices first
    filename = "./bifiltrations/" + name + ".txt"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "w") as f:
        f.write("--datatype bifiltration\n")
        f.write("--xlabel " + xlabel + "\n")
        f.write("--ylabel " + ylabel + "\n\n")
    
        for c in nx.enumerate_all_cliques(G):
            line = " ".join(map(str, c)) + " ; "
            new_vals = np.maximum.reduce([vert_vals[v] for v in c])
            line +=  " ".join(map(str, new_vals)) + "\n"
            f.write(line)

In [6]:
def hks(evals, t, vertex):
    # Computes the heat kernel signature of a graph with parameter t, given the eigendecomposition in evals.
    return sum([np.exp(-t * lam) * psi[vertex] * psi[vertex] for (lam, psi) in evals]).real


# Load graphs from file after being downloaded
# Compute the vertex features needed
graphs = {}
for e in endings:
    graph_list = nx.read_graph6("data/sr"+e+".g6")
    graphs[e] = []
    random = np.random.rand(int(e[:2])) 
    for G in graph_list:
        edecomp = np.linalg.eig(nx.normalized_laplacian_matrix(G).toarray())
        transp = sorted(zip(edecomp[0], edecomp[1]), key=lambda t: t[0])
        hks_G = hks(transp, 1, range(len(transp)))
        graphs[e].append({"graph": G, "hks" : hks_G, "random" : random})

In [7]:
# Compute the persistent homology of each graph, via the rivet_console program added to PATH
H0_persistence_modules = []
H1_persistence_modules = []
for e in endings:
    e_H0_pers_modules = []
    e_H1_pers_modules = []
    for (i, G) in enumerate(graphs[e]):
        filename = e + "_" + str(i)
        build_filtration( list(zip(G["hks"], G["random"])) , G["graph"], filename, "hks", "random")
        os.system("rivet_console ./bifiltrations/{}.txt ./bifiltrations/{}.rivet  --homology 0 --xbins 100 --ybins 100".format(filename, filename + "_0"))
        os.system("rivet_console ./bifiltrations/{}.txt ./bifiltrations/{}.rivet  --homology 1 --xbins 100 --ybins 100".format(filename, filename + "_1"))
        with open("./bifiltrations/{}.rivet".format(filename + "_0"), 'rb') as f:
            computed_data = f.read()
            e_H0_pers_modules.append(computed_data)
        with open("./bifiltrations/{}.rivet".format(filename + "_1"), 'rb') as f:
            computed_data = f.read()
            e_H1_pers_modules.append(computed_data)
    H0_persistence_modules.append(e_H0_pers_modules)
    H1_persistence_modules.append(e_H1_pers_modules)

In [ ]:
import matching_distance

# Now compute the accuracy to which the fibered barcodes on a 10 x 10 grid of lines
# distinguish pairs of graphs in the same collection.

# This might take a few hours to run, depending on your computer. Hera is written in python and is very slow.

# distance threshold to declare two barcodes distinct
eps = 1e-8

H0_accuracy = []
for (i, e) in enumerate(endings):
    total = 0
    correct = 0
    for (j1, P1) in enumerate(H0_persistence_modules[i]):
        for j2 in range(j1):
            total+=1
            P2 = H0_persistence_modules[i][j2]
            if matching_distance.matching_distance(P1, P2, grid_size=10, normalize=True) > eps:
                correct += 1
    H0_accuracy.append( correct / total )

H1_accuracy = []
for (i, e) in enumerate(endings):
    total = 0
    correct = 0
    for (j1, P1) in enumerate(H1_persistence_modules[i]):
        for j2 in range(j1):
            total+=1
            P2 = H1_persistence_modules[i][j2]
            if matching_distance.matching_distance(P1, P2, grid_size=10, normalize=True) > eps:
                correct += 1
    H1_accuracy.append( correct / total )

In [9]:
# Print the results of the previous computation.
print(H0_accuracy)
print(H1_accuracy)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
